# Import Subroutines and Settings

# *Procedure* for building " *The df_ib DataFrame from ib770 & ib970 combined with meta_data*

In [1]:
import sys
print(sys.executable)
print("note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.")

/home/bhuns/miniconda3/bin/python
note: THIS IS THE DIRECTORY PYTHON IS WORKING IN.


In [2]:
# Imports required for Loading, sorting .csx files to create specific data sets ie mrn inbody readings. 
%run ./sys_funcs.py              # loads all the def functions in sys_funcs.py into memory
#import sys_funcs                 # gives access to these def function digitalform that are in memory
from pathlib import Path
import csv
import pandas as pd
import numpy as np
import tkinter as tk
import pickle
from pathlib import Path
import csv
import os
import sys
from datetime import datetime
from datetime import time
from sys_funcs import read_csv_to_array
from sys_funcs import clean_wsl_path
from sys_funcs import array_to_dt_row_dict
from sys_funcs import make_blnk_update_row_dict
from sys_funcs import transpose_csv_to_col_dict
#from sys_funcs import update_values_with_config, get_update_result
from sys_funcs import transfer_updates
from sys_funcs import get_dtv_range
from sys_funcs import universal_import
from sys_funcs import parse_inbody_timestamp
from sys_funcs import build_lut
from sys_funcs import extract_a_column_as_df
from sys_funcs import extract_multicolumns_as_df
from sys_funcs import validate_and_sort_timestamps
from sys_funcs import extract_and_filter_by_time_window
from sys_funcs import read_file_dual_path
from sys_funcs import write_file_dual_path
from sys_funcs import asc_to_csv_cnv
from collections.abc import Mapping
import re
#from sys_funcs import 

In [3]:
# set print rows  This worksheet sets maximum # of rows printed
pd.set_option('display.max_rows', 1000)  # Adjust the number of rows to display
# pd.reset_option('display.max_rows')  
print('print set to 1000 rows max' )

print set to 1000 rows max


In [4]:
print("NOTE: timestamp = Test Date / Time does not work  use computed time stamp")


NOTE: timestamp = Test Date / Time does not work  use computed time stamp


# This is The worksheet that adds new data to the main data [m_ib_tst]


## def function to fill media

In [5]:
def fill_ib_leading_columns(df):
    """
    Fills the 5 leading operator columns for InBody datasets:
      - timestamp  ← parsed from 'Test Date / Time' (YYYYMMDDHHMMSS)
      - dtv        ← days since 1900‑01‑01
      - ib_id      ← 'mrn' if test time 03:00–23:59, else 'eve'
      - cls        ← NaN
      - cmmnts     ← NaN
    """

    df = df.copy()

    # --- 1. timestamp (correct parsing) ------------------------
    df['timestamp'] = pd.to_datetime(
        df['Test Date / Time'].astype(str),
        format="%Y%m%d%H%M%S",
        errors="coerce"
    )

    # --- 2. dtv: days since 1900‑01‑01 -------------------------
    origin = pd.Timestamp("1900-01-01")
    df['dtv'] = (df['timestamp'] - origin).dt.days

    # --- 3. ib_id classification -------------------------------
    def classify_ib_id(ts):
        if pd.isna(ts):
            return np.nan
        hour = ts.hour
        return "mrn" if 3 <= hour <= 23 else "eve"

    df['ib_id'] = df['timestamp'].apply(classify_ib_id)

    # --- 4. cls ------------------------------------------------
    df['cls'] = np.nan

    # --- 5. cmmnts ---------------------------------------------
    df['cmmnts'] = np.nan

    return df


In [6]:
def sort_by_timestamp(df):
    """
    Sorts an InBody dataframe by the 'timestamp' column
    in ascending chronological order.
    """
    df = df.copy()
    df = df.sort_values(by='timestamp', ascending=True)
    df = df.reset_index(drop=True)
    return df


In [7]:
def remove_ib_duplicates(df, subset_cols=None):
    """
    Removes duplicate InBody rows based on key identifying columns.
    Default behavior: remove duplicates based on ['ID', 'timestamp'].
    """
    df = df.copy()

    # Default duplicate definition
    if subset_cols is None:
        subset_cols = ['timestamp']
        # subset_cols = ['ID', 'timestamp']
    # Remove duplicates, keeping the first occurrence
    df = df.drop_duplicates(subset=subset_cols, keep='first')

    # Reset index for cleanliness
    df = df.reset_index(drop=True)

    return df


In [23]:
# this def function STRIPS COL NMBRS  column #s from col heads so_they match m_ib_tst col_names
def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

## Read latest [m_ib_tst] from xl and make a data frame

In [9]:
#this is the worksheet for bringing in the main data files from Excel

df_m_ib_tst = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv")
#print("df_m_ib_tst")
#verify print(df_m_ib_tst.columns.tolist())  # list form wo meta
#verify print(df_m_ib_tst)  # list form wo meta


## Fill in the meta daily columns by reading the data in the rest of the columns >>> **df_m_ib_tst**

In [14]:
# fill in meta col nmns
df_m_ib_tst = fill_ib_leading_columns(df_m_ib_tst)
print("meta cols filled")
df_m_ib_tst

meta cols filled


,dtv,timestamp,cls,ib_id,cmmnts,Name,ID,Height,Date of Birth,Age,...,Weight_Z score,50kHz-Whole Body Phase Angle_T score,50kHz-Whole Body Phase Angle_Z score,TBW/WT_T Score,TBW/WT_Z Score,SMI(SMM/Wt)_T score,SMI(SMM/Wt)_Z score,ECM/BCM_T Score,ECM/BCM Z Score,Unnamed: 250
0,46002,2025-12-13 05:49:21,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,46003,2025-12-14 23:48:14,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46006,2025-12-17 07:50:00,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46007,2025-12-18 08:16:04,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46005,2025-12-16 23:38:09,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46004,2025-12-15 08:00:38,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46001,2025-12-12 22:17:13,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,46008,2025-12-19 08:07:29,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,46003,2025-12-14 09:03:36,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,46007,2025-12-18 21:27:47,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Test the calculations of the timestamp DTV

In [15]:
# 
df_m_ib_tst[['Test Date / Time', 'timestamp',"dtv"]].head(20)


,Test Date / Time,timestamp,dtv
0,20251213054921,2025-12-13 05:49:21,46002
1,20251214234814,2025-12-14 23:48:14,46003
2,20251217075000,2025-12-17 07:50:00,46006
3,20251218081604,2025-12-18 08:16:04,46007
4,20251216233809,2025-12-16 23:38:09,46005
5,20251215080038,2025-12-15 08:00:38,46004
6,20251212221713,2025-12-12 22:17:13,46001
7,20251219080729,2025-12-19 08:07:29,46008
8,20251214090336,2025-12-14 09:03:36,46003
9,20251218212747,2025-12-18 21:27:47,46007


## Sort the data frame on the basis of timestamp you >> **df_m_ib_tst_s**

In [17]:
df_m_ib_tst_s = sort_by_timestamp(df_m_ib_tst)

In [18]:
# verify 
df_m_ib_tst_s[['Test Date / Time', 'timestamp',"dtv"]].head(20)


,Test Date / Time,timestamp,dtv
0,20250922225826,2025-09-22 22:58:26,45920
1,20251211092610,2025-12-11 09:26:10,46000
2,20251211092610,2025-12-11 09:26:10,46000
3,20251211092610,2025-12-11 09:26:10,46000
4,20251212084231,2025-12-12 08:42:31,46001
5,20251212084231,2025-12-12 08:42:31,46001
6,20251212084231,2025-12-12 08:42:31,46001
7,20251212221713,2025-12-12 22:17:13,46001
8,20251212221713,2025-12-12 22:17:13,46001
9,20251212221713,2025-12-12 22:17:13,46001


In [20]:
#verify 
df_m_ib_tst_s[["dtv","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

,dtv,SMM (Skeletal Muscle Mass),Weight,BMR (Basal Metabolic Rate),ECW/TBW
0,45920,98.3,262.4,2151,0.413
1,46000,99.4,260.1,2158,0.410
2,46000,99.4,260.1,2158,0.410
3,46000,99.4,260.1,2158,0.410
4,46001,101.2,259.2,2186,0.409
5,46001,101.2,259.2,2186,0.409
6,46001,101.2,259.2,2186,0.409
7,46001,101.6,262.3,2208,0.415
8,46001,101.6,262.3,2208,0.415
9,46001,101.6,262.3,2208,0.415


## Remove duplicates [...xd]  Retaining the first of a duplicate pair  >>> **df_m_ib_tst_s_xd**

In [21]:
df_m_ib_tst_s_xd  =  remove_ib_duplicates(df_m_ib_tst_s, subset_cols=None)
# verify
df_m_ib_tst_s_xd

,dtv,timestamp,cls,ib_id,cmmnts,Name,ID,Height,Date of Birth,Age,...,Weight_Z score,50kHz-Whole Body Phase Angle_T score,50kHz-Whole Body Phase Angle_Z score,TBW/WT_T Score,TBW/WT_Z Score,SMI(SMM/Wt)_T score,SMI(SMM/Wt)_Z score,ECM/BCM_T Score,ECM/BCM Z Score,Unnamed: 250
0,45920,2025-09-22 22:58:26,NaN,mrn,NaN,<bdailyn>,bdailyn,6ft 06.0in,1939.02.23.,86,...,2.5,-5.6,-2.6,-1.4,-0.3,-1.9,-0.6,8.0,4.6,NaN
1,46000,2025-12-11 09:26:10,NaN,mrn,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,46001,2025-12-12 08:42:31,NaN,mrn,NaN,251211-1,6ft 06.0in,1939.02.23.,M,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46001,2025-12-12 22:17:13,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46002,2025-12-13 05:49:21,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,46003,2025-12-14 09:03:36,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,46003,2025-12-14 23:48:14,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,46004,2025-12-15 08:00:38,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,46004,2025-12-15 23:33:24,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,46005,2025-12-16 06:48:08,NaN,mrn,NaN,91725,6ft 06.0in,1939.02.23.,F,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Slicing the fully processed data frame [df_m_ib_tst_s_xd] to display columns of interest  

In [22]:
#verify 
df_m_ib_tst_s_xd[["dtv","SMM (Skeletal Muscle Mass)" , "Weight" , "BMR (Basal Metabolic Rate)" , "ECW/TBW"]].head(20)

,dtv,SMM (Skeletal Muscle Mass),Weight,BMR (Basal Metabolic Rate),ECW/TBW
0,45920,98.3,262.4,2151,0.413
1,46000,99.4,260.1,2158,0.410
2,46001,101.2,259.2,2186,0.409
3,46001,101.6,262.3,2208,0.415
4,46002,102.3,260.0,2203,0.410
5,46003,99.2,258.0,2148,0.407
6,46003,99.2,261.1,2160,0.414
7,46004,99.9,258.3,2155,0.406
8,46004,103.4,260.0,2230,0.413
9,46005,98.3,257.0,2129,0.406


# The stored data is loaded check and verified and ready to receive new data

In [111]:
df_m_ib_tst_nms = df_m_ib_tst.columns
#verify 
df_m_ib_tst_nms

Index(['dtv', 'timestamp', 'cls', 'ib_id', 'cmmnts', 'Name', 'ID', 'Height',
       'Date of Birth', 'Age',
       ...
       'Weight_Z score', '50kHz-Whole Body Phase Angle_T score',
       '50kHz-Whole Body Phase Angle_Z score', 'TBW/WT_T Score',
       'TBW/WT_Z Score', 'SMI(SMM/Wt)_T score', 'SMI(SMM/Wt)_Z score',
       'ECM/BCM_T Score', 'ECM/BCM Z Score', 'Unnamed: 250'],
      dtype='object', length=325)

## Read the latest data from 770 and 970 in the data folder of repo

### ib77:  This segment reads the data from the Excel file_ib77 and computes the names of the column heads, strips them of numbers, puts them in a List, and records them in a pickle. *ib77_raw_nms*

In [112]:
# loads the new from the 770
ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)

✅ Loaded 091725_20251225081906.csv with utf-8
✅ Loaded 091725_20251213054921.csv with utf-8
✅ Loaded 091725_20251214234814.csv with utf-8
✅ Loaded 091725_20251223231059.csv with utf-8
✅ Loaded 091725_20251217075000.csv with utf-8
✅ Loaded 091725_20251218081604.csv with utf-8
✅ Loaded 091725_20251221071026.csv with utf-8
✅ Loaded 091725_20251216233809.csv with utf-8
✅ Loaded 091725_20251215080038.csv with utf-8
✅ Loaded 091725-1_20251212221713.csv with utf-8
✅ Loaded 091725_20251219080729.csv with utf-8
✅ Loaded 091725_20251221205127.csv with utf-8
✅ Loaded 091725_20251214090336.csv with utf-8
✅ Loaded 091725_20251218212747.csv with utf-8
✅ Loaded 091725_20251224080342.csv with utf-8
✅ Loaded 091725_20251222221501.csv with utf-8
✅ Loaded 091725_20251219232758.csv with utf-8
✅ Loaded 091725_20251215233324.csv with utf-8
✅ Loaded 091725_20251224234318.csv with utf-8
✅ Loaded 091725_20251216064808.csv with utf-8
✅ Loaded 091725-1_20251211092610.csv with utf-8
✅ Loaded 091725_20251223082717

In [114]:
ib77_raw.columns

Index(['1. Name', '2. ID', '3. Height', '4. Date of Birth', '5. Age',
       '6. Mobile Number', '7. Phone Number', '8. Zip Code', '9. Address',
       '10. E-mail',
       ...
       '143. R/Ht', '144. Xc/Ht', '145. HGS of Left Arm 1st',
       '146. HGS of Left Arm 2nd', '147. HGS of Right Arm 1st',
       '148. HGS of Right Arm 2nd', '149. HGS/WT', 'Unnamed: 149',
       'source_file', 'encoding_used'],
      dtype='object', length=152)

In [118]:
# strip numbers off So that the identical names in ib77 are the same as ib97
ib77_raw_nms = strip_col_numbers(ib77_raw)
ib77_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'TBW (Total Body Water)',
 'ICW (Intracellular Water)',
 'ECW (Extracellular Water)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'LBM (Lean Body Mass)',
 'SMM (Skeletal Muscle Mass)',
 'BMI (Body Mass Index)',
 'PBF (Percent Body Fat)',
 'LBM of Right Arm',
 'LBM% of Right Arm',
 'LBM of Left Arm',
 'LBM% of Left Arm',
 'LBM of Trunk',
 'LBM% of Trunk',
 'LBM of Right Leg',
 'LBM% of Right Leg',
 'LBM of Left Leg',
 'LBM% of Left Leg',
 'Leg Lean Mass',
 'TBW of Right Arm',
 'TBW of Left Arm',
 'TBW of Trunk',
 'TBW of Right Leg',
 'TBW of Left Leg',
 'ICW of Right Arm',
 'ICW of Left Arm',
 'ICW of Trunk',
 'ICW of Right Leg',
 'ICW of Left Leg',
 'ECW of Right Arm',
 'ECW of Left Arm',
 'ECW of Trunk',
 'ECW of Right Leg',
 'ECW of Left Leg',
 'ECW/TBW',
 'ECW/TBW of Right Arm',
 'ECW/TBW o

In [126]:
# Load the IB77 Columns without numbers to pickle 
with open("ib77_raw_nms.pkl", "wb") as f:
    pickle.dump(ib77_raw_nms, f)


In [135]:
# READ the IB77 Columns without numbers to pickle 
with open("ib77_raw_nms.pkl", "rb") as f:
    ib77_raw_nms = pickle.load(f)
ib77_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'TBW (Total Body Water)',
 'ICW (Intracellular Water)',
 'ECW (Extracellular Water)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'LBM (Lean Body Mass)',
 'SMM (Skeletal Muscle Mass)',
 'BMI (Body Mass Index)',
 'PBF (Percent Body Fat)',
 'LBM of Right Arm',
 'LBM% of Right Arm',
 'LBM of Left Arm',
 'LBM% of Left Arm',
 'LBM of Trunk',
 'LBM% of Trunk',
 'LBM of Right Leg',
 'LBM% of Right Leg',
 'LBM of Left Leg',
 'LBM% of Left Leg',
 'Leg Lean Mass',
 'TBW of Right Arm',
 'TBW of Left Arm',
 'TBW of Trunk',
 'TBW of Right Leg',
 'TBW of Left Leg',
 'ICW of Right Arm',
 'ICW of Left Arm',
 'ICW of Trunk',
 'ICW of Right Leg',
 'ICW of Left Leg',
 'ECW of Right Arm',
 'ECW of Left Arm',
 'ECW of Trunk',
 'ECW of Right Leg',
 'ECW of Left Leg',
 'ECW/TBW',
 'ECW/TBW of Right Arm',
 'ECW/TBW o

### ib97:  This segment Reads the data from the excel file_ib97 and computes names of the column head, strips them of numbers and Puts them in a list and records them in a pickle. *ib97_raw_nms*

In [131]:
# loads the new from the 970
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="*"
)

✅ Loaded 251201-1_20251207201219.csv with ISO-8859-1
✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 251201-1_20251205081353.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 251201-1_20251206072118.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loaded 091725-1_20251108074302.csv with ISO-8859-1
✅ Loaded 091725-1_20251028225751.csv with ISO-

In [130]:
ib97_raw.columns

Index(['1. Name', '2. ID', '3. Height', '4. Date of Birth', '5. Age',
       '6. Mobile Number', '7. Phone Number', '8. Zip Code', '9. Address',
       '10. E-mail',
       ...
       '248. HGS of Left Arm 1st', '249. HGS of Left Arm 2nd',
       '250. HGS of Right Arm 1st', '251. HGS of Right Arm 2nd', '252. HGS/WT',
       'Unnamed: 252', '152 50KH ', '-33.5', 'jb', 'bjh970'],
      dtype='object', length=533)

In [132]:
# strip numbers off So that the identical names in ib97 are the same as ib97
ib97_raw_nms = strip_col_numbers(ib97_raw)
ib97_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'Lower Limit (Weight Normal Range)',
 'Upper Limit (Weight Normal Range)',
 'TBW (Total Body Water)',
 'Lower Limit (TBW Normal Range)',
 'Upper Limit (TBW Normal Range)',
 'ICW (Intracellular Water)',
 'Lower Limit (ICW Normal Range)',
 'Upper Limit (ICW Normal Range)',
 'ECW (Extracellular Water)',
 'Lower Limit (ECW Normal Range)',
 'Upper Limit (ECW Normal Range)',
 'Protein',
 'Lower Limit (Protein Normal Range)',
 'Upper Limit (Protein Normal Range)',
 'Minerals',
 'Lower Limit (Minerals Normal Range)',
 'Upper Limit (Minerals Normal Range)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'Lower Limit (BFM Normal Range)',
 'Upper Limit (BFM Normal Range)',
 'FFM (Fat Free Mass)',
 'SMM (Skeletal Muscle Mass)',
 'Lower Limit (SMM Normal Range)',
 'Upper Limit (SMM Normal Range)',
 'BMI (Body

In [136]:
# Load the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "wb") as f:
    pickle.dump(ib97_raw_nms, f)


In [137]:
# READ the IB97 Columns without numbers to pickle 
with open("ib97_raw_nms.pkl", "rb") as f:
    ib97_raw_nms = pickle.load(f)
ib97_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'Lower Limit (Weight Normal Range)',
 'Upper Limit (Weight Normal Range)',
 'TBW (Total Body Water)',
 'Lower Limit (TBW Normal Range)',
 'Upper Limit (TBW Normal Range)',
 'ICW (Intracellular Water)',
 'Lower Limit (ICW Normal Range)',
 'Upper Limit (ICW Normal Range)',
 'ECW (Extracellular Water)',
 'Lower Limit (ECW Normal Range)',
 'Upper Limit (ECW Normal Range)',
 'Protein',
 'Lower Limit (Protein Normal Range)',
 'Upper Limit (Protein Normal Range)',
 'Minerals',
 'Lower Limit (Minerals Normal Range)',
 'Upper Limit (Minerals Normal Range)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'Lower Limit (BFM Normal Range)',
 'Upper Limit (BFM Normal Range)',
 'FFM (Fat Free Mass)',
 'SMM (Skeletal Muscle Mass)',
 'Lower Limit (SMM Normal Range)',
 'Upper Limit (SMM Normal Range)',
 'BMI (Body

In [127]:
# Load the IB77 Columns without numbers to pickle 
with open("ib77_raw_nms.pkl", "rb") as f:
    ib77_raw_nms = pickle.load(f)
ib77_raw_nms

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'TBW (Total Body Water)',
 'ICW (Intracellular Water)',
 'ECW (Extracellular Water)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'LBM (Lean Body Mass)',
 'SMM (Skeletal Muscle Mass)',
 'BMI (Body Mass Index)',
 'PBF (Percent Body Fat)',
 'LBM of Right Arm',
 'LBM% of Right Arm',
 'LBM of Left Arm',
 'LBM% of Left Arm',
 'LBM of Trunk',
 'LBM% of Trunk',
 'LBM of Right Leg',
 'LBM% of Right Leg',
 'LBM of Left Leg',
 'LBM% of Left Leg',
 'Leg Lean Mass',
 'TBW of Right Arm',
 'TBW of Left Arm',
 'TBW of Trunk',
 'TBW of Right Leg',
 'TBW of Left Leg',
 'ICW of Right Arm',
 'ICW of Left Arm',
 'ICW of Trunk',
 'ICW of Right Leg',
 'ICW of Left Leg',
 'ECW of Right Arm',
 'ECW of Left Arm',
 'ECW of Trunk',
 'ECW of Right Leg',
 'ECW of Left Leg',
 'ECW/TBW',
 'ECW/TBW of Right Arm',
 'ECW/TBW o

# This is the old version that had some problems The one above is just a copy of the 97 which was proven to work

In [128]:
# loads the new from the 970
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="*"
)
ib97_raw.columns

✅ Loaded 251201-1_20251207201219.csv with ISO-8859-1
✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 251201-1_20251205081353.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 251201-1_20251206072118.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loaded 091725-1_20251108074302.csv with ISO-8859-1
✅ Loaded 091725-1_20251028225751.csv with ISO-

Index(['1. Name', '2. ID', '3. Height', '4. Date of Birth', '5. Age',
       '6. Mobile Number', '7. Phone Number', '8. Zip Code', '9. Address',
       '10. E-mail',
       ...
       '248. HGS of Left Arm 1st', '249. HGS of Left Arm 2nd',
       '250. HGS of Right Arm 1st', '251. HGS of Right Arm 2nd', '252. HGS/WT',
       'Unnamed: 252', '152 50KH ', '-33.5', 'jb', 'bjh970'],
      dtype='object', length=533)

In [102]:
ib97_numbr_col = ib97_raw.columns.astype(str).tolist
print(ib97_numbr_col)

<bound method IndexOpsMixin.tolist of Index(['1. Name', '2. ID', '3. Height', '4. Date of Birth', '5. Age',
       '6. Mobile Number', '7. Phone Number', '8. Zip Code', '9. Address',
       '10. E-mail',
       ...
       '248. HGS of Left Arm 1st', '249. HGS of Left Arm 2nd',
       '250. HGS of Right Arm 1st', '251. HGS of Right Arm 2nd', '252. HGS/WT',
       'Unnamed: 252', '152 50KH ', '-33.5', 'jb', 'bjh970'],
      dtype='object', length=533)>


In [103]:
ib97_raw.columns[ib97_raw.columns.duplicated()]


Index([], dtype='object')

In [90]:
# strip numbers off So that the identical names in ib97 are the same as ib77
ib97_raw_nms = strip_col_numbers(ib97_raw)

['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'Lower Limit (Weight Normal Range)',
 'Upper Limit (Weight Normal Range)',
 'TBW (Total Body Water)',
 'Lower Limit (TBW Normal Range)',
 'Upper Limit (TBW Normal Range)',
 'ICW (Intracellular Water)',
 'Lower Limit (ICW Normal Range)',
 'Upper Limit (ICW Normal Range)',
 'ECW (Extracellular Water)',
 'Lower Limit (ECW Normal Range)',
 'Upper Limit (ECW Normal Range)',
 'Protein',
 'Lower Limit (Protein Normal Range)',
 'Upper Limit (Protein Normal Range)',
 'Minerals',
 'Lower Limit (Minerals Normal Range)',
 'Upper Limit (Minerals Normal Range)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'Lower Limit (BFM Normal Range)',
 'Upper Limit (BFM Normal Range)',
 'FFM (Fat Free Mass)',
 'SMM (Skeletal Muscle Mass)',
 'Lower Limit (SMM Normal Range)',
 'Upper Limit (SMM Normal Range)',
 'BMI (Body

In [97]:
ib97_raw_nms.columns[ib97_raw_nms.columns.duplicated()]


AttributeError: 'list' object has no attribute 'columns'

In [89]:
# Now we'll make it into a data frame so that we will print it in a column which obey list [numbers separated by commas]
df_ib97_raw_nms = pd.DataFrame(ib97_raw_nms)

# Load to pickle file in the working directory
df_ib97_raw_nms.to_pickle("df_ib97_raw_nms.pkl")

# For verification the picket file and print its value.
df_ib97_raw_nms = pd.read_pickle("df_ib97_raw_nms.pkl")

# verify 
print(ib97_raw_nms)
# verify 
print("df_ib97_raw_nms \n",df_ib97_raw_nms)

['Name', 'ID', 'Height', 'Date of Birth', 'Age', 'Mobile Number', 'Phone Number', 'Zip Code', 'Address', 'E-mail', 'Date of Registration', 'Memo', 'Test Date / Time', 'Weight', 'Lower Limit (Weight Normal Range)', 'Upper Limit (Weight Normal Range)', 'TBW (Total Body Water)', 'Lower Limit (TBW Normal Range)', 'Upper Limit (TBW Normal Range)', 'ICW (Intracellular Water)', 'Lower Limit (ICW Normal Range)', 'Upper Limit (ICW Normal Range)', 'ECW (Extracellular Water)', 'Lower Limit (ECW Normal Range)', 'Upper Limit (ECW Normal Range)', 'Protein', 'Lower Limit (Protein Normal Range)', 'Upper Limit (Protein Normal Range)', 'Minerals', 'Lower Limit (Minerals Normal Range)', 'Upper Limit (Minerals Normal Range)', 'DLM (Dry Lean Mass)', 'BFM (Body Fat Mass)', 'Lower Limit (BFM Normal Range)', 'Upper Limit (BFM Normal Range)', 'FFM (Fat Free Mass)', 'SMM (Skeletal Muscle Mass)', 'Lower Limit (SMM Normal Range)', 'Upper Limit (SMM Normal Range)', 'BMI (Body Mass Index)', 'Lower Limit (BMI Normal

In [84]:
# The column name originally included numbers  and that # was stripped off in "ib97_raw_nms" so The column names were reassigned with No numbers
ib97_raw.columns=ib97_raw_nms
#verify
#
ib77_raw.columns


Index(['1. Name', '2. ID', '3. Height', '4. Date of Birth', '5. Age',
       '6. Mobile Number', '7. Phone Number', '8. Zip Code', '9. Address',
       '10. E-mail',
       ...
       '241. Weight_T score', '242. Weight_Z score',
       '243. 50kHz-Whole Body Phase Angle_T score',
       '244. 50kHz-Whole Body Phase Angle_Z score', '245. TBW/WT_T Score',
       '246. TBW/WT_Z Score', '247. SMI(SMM/Wt)_T score',
       '248. SMI(SMM/Wt)_Z score', '249. ECM/BCM_T Score',
       '250. ECM/BCM Z Score'],
      dtype='object', length=398)

In [85]:
ib77_raw.columns[ib77_raw.columns.duplicated()]


Index([], dtype='object')

In [92]:
ib77_raw-nms[ib77_raw-nms.columns.duplicated()]


NameError: name 'nms' is not defined

# The following data are loaded and ready to be consolidated into an updated Total data data frame "m_bi_tst" from :old "m_bi_tst" and new"ib97_raw",new "ib77_raw"

### This will load the ib77 into the accumulated data

In [43]:
# Add any missing columns to IB77_raw
for col in df_m_ib_tst.columns:
    if col not in ib77_raw.columns:
        ib77_raw[col] = None

# Reorder ib77_raw columns to match df_m_ib_tst
ib77_raw = ib77_raw[df_m_ib_tst.columns]


In [ ]:
df_m_ib_tst = pd.concat([df_m_ib_tst, ib77_raw], ignore_index=True)
df_m_ib_tst

In [45]:
df_m_ib_tst.columns[df_m_ib_tst.columns.duplicated()]


Index([], dtype='object')

In [93]:
ib77_raw.columns[ib77_raw.columns.duplicated()]


Index([], dtype='object')

### This will load the ib97 into the accumulated data

In [ ]:
# Add any missing columns to IB97_raw
for col in m_ib_tst.columns:
    if col not in ib97_raw.columns:
        ib97_raw[col] = None
# Reorder IB97_raw columns to match m_ib_tst
ib97_raw = ib97_raw[m_ib_tst.columns]


In [ ]:
m_ib_tst = pd.concat([m_ib_tst, ib97_raw], ignore_index=True)
m_ib_tst

# load latest [m_ib_tst] to xl

In [ ]:
# load latest [m_ib_tst] from_xl
folder_path = "/home/bhuns/JL_2/data/ib_tst/m_ib_tst.csv"
m_ib_tst.to_csv(folder_path, index=False)
# verify
#
m_ib_tst

# Everything below this line is residual the days the template was developed and completed. Erase it when everything using the template is completed
# ================================================================================================================================================================

# Startup only Create  **df_ib_fls_tmplt**  ie  *[COL_NMS = meta+ib770dat + ib970dat]*

## Add the columns of df_77 & df_97 then delete Column names and add meta cols and Store as data frames in pickle called *df_ib_tst_nms*

In [ ]:
# this def function STRIPS COL NMBRS  column #s from col heads used only in making the files

def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

### ib77  This segment Reads the data from the excel file and computes names of the column head, strips them of numbers and Puts them in a data frame and records them in a pickle. *"df_ib77_raw_nms"*

In [ ]:
# loads the new from the 770
ib77_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib77",
    pattern="*"
)
# strip numbers off So that the identical names in IB77 are the same as IB97
ib77_raw_nms = strip_col_numbers(ib77_raw)
# Now we'll make it into a data frame so that we will print it in a column which obey list [numbers separated by commas]
df_ib77_raw_nms = pd.DataFrame(ib77_raw_nms)

# Load to pickle file in the working directory
df_ib77_raw_nms.to_pickle("df_ib77_raw_nms.pkl")

# For verification the picket file and print its value.
df_ib77_raw_nms = pd.read_pickle("df_ib77_raw_nms.pkl")

# verify 
# print(ib77_raw_nms)
# verify print("df_ib77_raw_nms \n",df_ib77_raw_nms)

### ib97 This segment Reads the data from the excel file and computes names of the column head, strips them of numbers and Puts them in a data frame and records them in a pickle. *"df_ib97_raw_nms"*

In [95]:
ib97_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib97",
    pattern="*"
)
# strip numbers off So that the identical names in IB97 are the same as IB97
ib97_raw_nms = strip_col_numbers(ib97_raw)

✅ Loaded 251201-1_20251207201219.csv with ISO-8859-1
✅ Loaded 091725-1_20251118042051.csv with ISO-8859-1
✅ Loaded 091725-1_20251023065307.csv with ISO-8859-1
✅ Loaded 091725-1_20250928235640.csv with ISO-8859-1
✅ Loaded 091725-1_20251025233431.csv with ISO-8859-1
✅ Loaded 091725-1_20251124075021.csv with ISO-8859-1
✅ Loaded 091725-1_20251021000847.csv with ISO-8859-1
✅ Loaded 091725-1_20251101213555.csv with ISO-8859-1
✅ Loaded 251201-1_20251205081353.csv with ISO-8859-1
✅ Loaded 091725-1_20251116230333.csv with ISO-8859-1
✅ Loaded 251201-1_20251206072118.csv with ISO-8859-1
✅ Loaded 091725-1_20251130055027.csv with ISO-8859-1
✅ Loaded 091725-1_20251019230750.csv with ISO-8859-1
✅ Loaded 091725-1_20251017091340.csv with ISO-8859-1
✅ Loaded 091725-1_20251002065435.csv with ISO-8859-1
✅ Loaded 091725-1_20251019074606.csv with ISO-8859-1
✅ Loaded 091725-1_20251118230931.csv with ISO-8859-1
✅ Loaded 091725-1_20251108074302.csv with ISO-8859-1
✅ Loaded 091725-1_20251028225751.csv with ISO-

In [96]:
ib97_raw_nms


['Name',
 'ID',
 'Height',
 'Date of Birth',
 'Age',
 'Mobile Number',
 'Phone Number',
 'Zip Code',
 'Address',
 'E-mail',
 'Date of Registration',
 'Memo',
 'Test Date / Time',
 'Weight',
 'Lower Limit (Weight Normal Range)',
 'Upper Limit (Weight Normal Range)',
 'TBW (Total Body Water)',
 'Lower Limit (TBW Normal Range)',
 'Upper Limit (TBW Normal Range)',
 'ICW (Intracellular Water)',
 'Lower Limit (ICW Normal Range)',
 'Upper Limit (ICW Normal Range)',
 'ECW (Extracellular Water)',
 'Lower Limit (ECW Normal Range)',
 'Upper Limit (ECW Normal Range)',
 'Protein',
 'Lower Limit (Protein Normal Range)',
 'Upper Limit (Protein Normal Range)',
 'Minerals',
 'Lower Limit (Minerals Normal Range)',
 'Upper Limit (Minerals Normal Range)',
 'DLM (Dry Lean Mass)',
 'BFM (Body Fat Mass)',
 'Lower Limit (BFM Normal Range)',
 'Upper Limit (BFM Normal Range)',
 'FFM (Fat Free Mass)',
 'SMM (Skeletal Muscle Mass)',
 'Lower Limit (SMM Normal Range)',
 'Upper Limit (SMM Normal Range)',
 'BMI (Body

In [ ]:
# Now we'll make it into a data frame so that we will print it in a column which obey list [numbers separated by commas]
df_ib97_raw_nms = pd.DataFrame(ib97_raw_nms)

# Load to pickle file in the working directory
df_ib97_raw_nms.to_pickle("df_ib97_raw_nms.pkl")

# For verification the picket file and print its value.
df_ib97_raw_nms = pd.read_pickle("df_ib97_raw_nms.pkl")

# verify 
# print(ib97_raw_nms)
# verify 
# print("df_ib97_raw_nms \n",df_ib97_raw_nms)

### Both the 77 and the 90 test datasets are convrted to dfs In order to determine *df_ib_tst_nms* Which is a single column data frame that all of the column names of the *df_ib_tst*

#### ✅ Concatenate → Preserve Order → Drop Duplicates

In [28]:
####  Concatenate → Preserve Order → Drop Duplicates mmmmmmmmmnnnnnnnllllll
# Concatenate in the required order
df_ib_tst_nms = pd.concat(
    [df_ib77_raw_nms, df_ib97_raw_nms],
    axis=0,
    ignore_index=True
)
# print(df_ib_tst_nms)
# Remove duplicate rows, keeping the first occurrence (from df_ib77_raw_nms)
df_ib_tst_nms = df_ib_tst_nms.drop_duplicates(keep="first")
# verify 
# print(df_ib_tst_nms)

In [ ]:
# Verifies that the duplicates have been removed and that the list has been compacted and there are not duplicates of 770 and 970 Col
df_ib_tst_nms[df_ib_tst_nms.duplicated()]

### *df_ib_tst_nms* must be converted to a simple list in order to made equal to the column heads of df_tst

In [ ]:
# The large test data frame that will be used to accept values of each from both 770 N 970 will be built on the basis of the df_ib_tst_nms, but this data bugs be in form of a strings.
# df['colname'].astype(str).tolist       # sign This is the sample given from copilot that is modified below for my situation 

ib_tst_lst = df_ib_tst_nms[0].astype(str).tolist()
# verify 
# print(ib_tst_lst)

In [ ]:
# this makes the template for the ib_tst data For both the 770 and the 970 It is made up of a 770 and a 970 with duplicates eliminated in a list of strings

df_ib_tst = pd.DataFrame(
    data = [[""] * len(df_ib_tst_lst)] * 2,
    columns = ib_tst_lst
)
#df_ib_tst_nms
df_ib_tst

In [ ]:
# Tests 
df_ib_tst["ID"]


In [ ]:
df_ib_tst.columns


In [ ]:
df_ib_tst["ID"]

# The following is the procedure to edit_ This structure of ” ib_tst”

## data frame of the main data storage_“ib_tst"

###  Make the edits to the structure of the main data spreadsheet that represents structure

### Then the CSV file into the worksheet to display the changes in Jupiter lab

In [ ]:
df_loaded = pd.read_csv("/home/bhuns/JL_2/data/ib_tst/ib_tst.csv")
df_loaded.columns

In [ ]:
folder_path = "/home/bhuns/JL_2/data/ib_tst/ib_tst.csv"
df_ib_tst.to_csv(folder_path, index=False)
df_ib_tst

In [ ]:
df_loaded["Age"]



In [ ]:
#==================================================================================================================== Oh

###


# Align the columns via index suggestions from Copilot to insert new data into the major data frame

Put the creation of a data frame into its own worksheet

# ==============================================================================================================================================

## Calculate the *df_ib_tst* column names from the beta column names added to the 770 column names to the 970 add column names

## Create *df_ib_fls_tmplt* using the *df_ib_fls* column names.

## Then save it to pickle as df_ib_fls_tmplt

## Then save it to df_ib_fls.csv  to start a new data set

# Import the ib770 & ib970 to **df_ib_fls** These will be fill in column by column using the column name as a reference and added row by row. 

## Import *df_ib_fls.csv*

##  Import *ib97.csv and remove numbers from* And apend to *df_ib_fls* if timestamp does not exist 

##  Import *ib77.csv*  And apend to *df_ib_fls* if timestamp does not exist 

## 

## 

In [ ]:
ib_fls_raw = universal_import(
    folder_path="/home/bhuns/JL_2/data/ib_fls",
    pattern="*"
)
df_ib_fls_raw = pd.DataFrame(ib_fls_raw)

# verify
# print( df_ib_fls_raw["2. ID"])


In [ ]:
# verify
# print( df_ib_fls_raw["2. ID"])

# df_col_nm_lst =["",""]
df_col_nm_lst = pd.DataFrame( df_ib_fls_raw.columns)
# verify
print(df_col_nm_lst)

# Remove **numbers** from col_nms   ok

In [ ]:
# STRIPS COL NMBRS  column #s from col heads

def strip_col_numbers(col_names):
    """
    Remove leading numbers and dots from column names.
    Example: '244. 50kHz-Whole Body Phase Angle_Z score' 
             -> '50kHz-Whole Body Phase Angle_Z score'
    """
    return [re.sub(r'^\d+\.\s*', '', name) for name in col_names]

# Run this for both the 970 and the 770 data imports
# col_names = C.columns
# df_ib_fls_raw_nm = pd.DataFrame(strip_col_numbers(col_names))
# verify
# print(df_ib_fls_raw_nm)

# Remove Duplicates so each col in 770 and 970 Have one unique Column In this overall *df_ib* dataframe   OK

In [ ]:
df_ib_fls_raw_nm = df_ib_fls_raw_nm.drop_duplicates(keep="first").reset_index(drop=True)


In [ ]:
# verify
# 
print(df_ib_fls_raw_nm)

In [ ]:
print(df_ib_fls_raw)